In [21]:
import escher
from escher import Builder
import cobra
from time import sleep

In [22]:
!ls ../../../../GEM-repos/ecoli/

iJO1366.Central metabolism.json iJO1366.xml
iJO1366.json


In [23]:
# Load the model into cobra
# Relative path to model file saved locally on Helen's computer, but no changes to the model since downloading it from BiGG
model = cobra.io.load_json_model('../../../../GEM-repos/ecoli/iJO1366.json')

In [24]:
model

Name,iJO1366
Memory address,7fb31077a4f0
Number of metabolites,1805
Number of reactions,2583
Number of genes,1367
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [25]:
model.metabolites.ac_e

Metabolite identifier,ac_e
Name,Acetate
Memory address,0x7fb311031370
Formula,C2H3O2
Compartment,e
In 2 reaction(s),"ACtex, EX_ac_e"


In [26]:
# Check what the default medium is
model.medium

{'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0}

In [27]:
# Run FBA for the default medium
glc_10_solution = model.optimize()
glc_10_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-2.775558e-17
RNTR2c2,0.026541,2.775558e-17


In [28]:
# Make a map with the solution for the default medium
glc_10_map = Builder(
    map_json = '../../../../GEM-repos/ecoli/iJO1366.Central metabolism.json',
    reaction_data = glc_10_solution.fluxes
)

In [29]:
# View the map
glc_10_map

Builder(reaction_data={'EX_cm_e': 0.0, 'EX_cmp_e': 0.0, 'EX_co2_e': 19.67522263566321, 'EX_cobalt2_e': -2.4559…

In [30]:
# Define medium with just a little bit of acetate available
# Note, this does not set the flux, just the limits on it
glc_ace_medium = model.medium
glc_ace_medium['EX_ac_e'] = 1

In [31]:
# Run FBA for the glucose + acetate medium
model.medium = glc_ace_medium
ace_1_solution = model.optimize()
ace_1_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,20.568568,0.000000e+00
EX_cobalt2_e,-0.000025,0.000000e+00
DM_4crsol_c,0.000225,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.026410,0.000000e+00
RNTR2c2,0.027269,2.775558e-17


In [32]:
# Make a map with the solution for the defulat medium
ace_1_map = Builder(
    map_json = '../../../../GEM-repos/ecoli/iJO1366.Central metabolism.json',
    reaction_data = ace_1_solution.fluxes
)

In [33]:
ace_1_map

Builder(reaction_data={'EX_cm_e': 0.0, 'EX_cmp_e': 0.0, 'EX_co2_e': 20.568568412027176, 'EX_cobalt2_e': -2.523…

In [34]:
# Get a list of the reactions that are different between the two solutions, with the difference between them
diff_reactions = {}
for reaction in model.reactions:
    if abs(glc_10_solution.fluxes[reaction.id] - ace_1_solution.fluxes[reaction.id]) > 1e-6:
        diff_reactions[reaction.id] = ace_1_solution.fluxes[reaction.id] - glc_10_solution.fluxes[reaction.id]

In [35]:
# Print the reaction ID, the reaction name, the flux in the glucose medium, and the flux in the glucose + acetate medium in the order of the difference between the two
for reaction in sorted(diff_reactions, key = lambda x: abs(diff_reactions[x])):
    print(reaction, model.reactions.get_by_id(reaction).name, glc_10_solution.fluxes[reaction], ace_1_solution.fluxes[reaction])

UDCPDPS Undecaprenyl diphosphate synthase 5.403044969998721e-05 5.551323594887681e-05
BMOCOS Bis-molybdenum cofactor synthase 0.00011984936115269137 0.00012313845065023166
BMOGDS1 Bis-molybdopterin guanine dinucleotide synthase (single GDP) 0.00011984936115269137 0.00012313845065023166
MOCOS Molybdenum cofactor synthase 0.00011984936115269137 0.00012313845065023166
BMOGDS2 Bis-molybdopterin guanine dinucleotide synthase 0.00011984936115269135 0.00012313845065023166
EX_mobd_e Molybdate exchange -0.0001267259638417802 -0.00013020377158917942
MOBDabcpp Molybdate transport via ABC system (periplasm) 0.0001267259638417802 0.00013020377158917942
MOBDtex Molybdate transport via diffusion (extracellular to periplasm) 0.0001267259638417802 0.00013020377158917942
FMNAT FMN adenylyltransferase 0.0002190689142382293 0.00022508093848363743
RBFK Riboflavin kinase 0.0002190689142382293 0.00022508093848363743
PDX5POi Pyridoxine 5'-phosphate oxidase 0.0002190689142381162 0.00022508093848362014
PDX5PS P

In [37]:
model.reactions.EX_glc__D_e.metabolites

{<Metabolite glc__D_e at 0x7fb3110762e0>: -1.0}

In [38]:
model.reactions.HEX1

Reaction identifier,HEX1
Name,Hexokinase (D-glucose:ATP)
Memory address,0x7fb2f5693940
Stoichiometry,atp_c + glc__D_c --> adp_c + g6p_c + h_c ATP C10H12N5O13P3 + D-Glucose --> ADP C10H12N5O10P2 + D-Glucose 6-phosphate + H+
GPR,b2388
Lower bound,0.0
Upper bound,1000.0


In [40]:
model.metabolites.g6p_c

Metabolite identifier,g6p_c
Name,D-Glucose 6-phosphate
Memory address,0x7fb2f4de8fa0
Formula,C6H11O9P
Compartment,c
In 12 reaction(s),"PGMT, TRE6PH, FFSD, GLCptspp, G6Pt6_2pp, G6PP, FRULYSDG, G6PDH2r, HEX1, PGI, TRE6PS, AB6PGH"


In [41]:
model.reactions.PGI

Reaction identifier,PGI
Name,Glucose-6-phosphate isomerase
Memory address,0x7fb2f58d6f40
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b4025
Lower bound,-1000.0
Upper bound,1000.0
